In [ ]:
!pip install transformers
!pip install datasets
!pip install randomdict

In [ ]:
import os
import random
import numpy as np
from randomdict import RandomDict

import torch
import torch.nn.functional as F
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForMaskedLM, DataCollatorWithPadding, DataCollatorForLanguageModeling

In [ ]:
#kaggle creation
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets init -p '/content/reddit_test_dataset'
!kaggle datasets create -p '/content/reddit_train_dataset'

!kaggle datasets init -p '/content/reddit'
!kaggle datasets create -p '/content/reddit_train_dataset'

In [ ]:
def tokenize_fn(element):
  sample = tokenizer(element['content'], truncation=True, return_special_tokens_mask=True)

  ids = np.array(sample['input_ids'])
  tokens_mask = np.array(sample['special_tokens_mask'])
  labels_test = sample['input_ids']
  attention_mask = sample['attention_mask']

  rand_array = np.random.rand(*ids.shape)
  mask_array = (rand_array<.15) * ~tokens_mask.astype(bool)

  mask_idx = np.where(mask_array)[0]
  rand_mask = np.random.rand(*mask_idx.shape)
  mask_positions = mask_idx[rand_mask<.8]
  mask_random_word = mask_idx[(.8<=rand_mask) & (rand_mask<.9)]

  ids[mask_positions] = tokenizer.mask_token_id
  for elemen in mask_random_word:
    ids[elemen] = random_vocab.random_value()

  ids = ids.tolist()

  return {'input_ids': ids, 'attention_mask': attention_mask, 'labels': labels_test}

In [ ]:
#dataset_creation
random_vocab = RandomDict(tokenizer.vocab)
del random_vocab['[CLS]'], random_vocab['[MASK]'], random_vocab['[PAD]'], random_vocab['[SEP]'], random_vocab['[UNK]']

dataset_train = load_dataset('reddit', split='train[:70%]')
tokenized_train = dataset_train.map(tokenize_fn, remove_columns=['author', 'body', 'id', 'normalizedBody', 'subreddit', 'summary', 'subreddit_id', 'content'], batched=False)
tokenized_train.save_to_disk('reddit_train_dataset')

dataset_test = load_dataset('reddit', split='train[30%:]')
tokenized_test = dataset_test.map(tokenize_fn, remove_columns=['author', 'body', 'id', 'normalizedBody', 'subreddit', 'summary', 'subreddit_id', 'content'], batched=False)
tokenized_test.save_to_disk('reddit_test_dataset')